### Import

In [51]:
%gui qt
from PyQt4 import QtGui, QtCore
from IPython.html import widgets
from IPython.display import display

import glob
import os
from shutil import copyfile, rmtree

## Documentation 

This notebook will allow to **deal** to a given number of folders, a set of files.

_examples:_

**    number of folder = 2**
  
    file_001 -> folder_1
    file_002 -> folder_2
    file_003 -> folder_1
    file_004 -> folder_2
    ..
    
    
**    number of folder = 3**

    file_001 -> folder_1
    file_002 -> folder_2
    file_003 -> folder_3
    file_004 -> folder_1
    file_005 -> folder_2
    file_006 -> folder_3
    file_007 -> folder_1
    ..
    

## select input folder

In [2]:
def gui_fname(dir=None):
    """Select a file via a dialog and returns the file name.
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [6]:
%gui qt
input_dname = gui_fname()

## select top output folder 

In [7]:
%gui qt
output_dname = gui_fname()

## select number of output folder

In [14]:
deal_coefficient_w = widgets.IntSlider(
    value=2,
    min=2,
    max=10.0,
    step=1,
    description='Deal Coefficient:',
    orientation='horizontal',
)
display(deal_coefficient_w)

In [46]:
deal_coefficient = deal_coefficient_w.value

## Perform calculations 

In [47]:
def create_log_file(folder = '', content = None):
    if content is None:
        return
    
    output_file_name = os.path.join(folder, 'log.txt')
    f = open(output_file_name, 'w')
    for _line in content:
        f.write(_line + "\n")
        
    f.close()
    

In [52]:
# retrieve list of files to deal
ext = ''

list_of_files_fits = glob.glob(input_dname + '/*.fits')
list_of_files_tif = glob.glob(input_dname + '/*.tif')

list_of_files = []
if len(list_of_files_fits) == len(list_of_files_tif):
    raise ValueError("Can not figure out the input files you want to work on!")
elif len(list_of_files_fits) > len(list_of_files_tif):
    list_of_files = list_of_files_fits
    ext = '.fits'
else:
    list_of_files = list_of_files_tif
    ext = '.tif'
    
# create output folders
list_output_folder = []
for index in range(deal_coefficient):    
    _folder_name = "folder_%.2d" %index
    list_output_folder.append(_folder_name)
    
for _folder in list_output_folder:
    _full_folder_name = os.path.join(output_dname, _folder)
    if os.path.isdir(_full_folder_name) and os.path.exists(_full_folder_name):
        rmtree(_full_folder_name)
    os.mkdir(_full_folder_name)

# move files to right folders
file_index = 0
folder_index = 0

log_renaming_table = ['#Renaming log']
log_renaming_table.append('#Input Folder: %s' %input_dname)
log_renaming_table.append('#Output Folder: %s' % output_dname)
log_renaming_table.append('#Deal coeficient: %d' %deal_coefficient)
log_renaming_table.append('#')
log_renaming_table.append('#old_file_name \tnew_folder_destination \tnew_file_name')

for _file in list_of_files:

    #figure out the new name
    _new_file_name = "file_%.4d" % file_index
    _new_file_name += ext
    _folder_name = list_output_folder[folder_index]
    _new_full_file_name = os.path.join(output_dname, _folder_name, _new_file_name)
    
    #mv file to right folder
    copyfile(_file, _new_full_file_name)
    
    _log = "%s \t%.2d \t%s" %(_file, folder_index, _new_file_name)
    log_renaming_table.append(_log)
    
    folder_index += 1
    if folder_index > (deal_coefficient-1):
        folder_index = 0
        file_index += 1

# output log file
create_log_file(folder = output_dname, content = log_renaming_table)
        
        